In [1]:
import numpy as np

In [2]:
rng = np.random.default_rng(20000)

In [3]:
dZ = np.array([[[ 0.00250992,  0.00250992,  0.00250992, -0.14958683,
          0.13703721,  0.00250992,  0.00250992],
        [ 0.0229403 ,  0.0229403 ,  0.0229403 ,  0.0229403 ,
         -0.1376418 ,  0.0229403 ,  0.0229403 ],
        [ 0.01493757,  0.01493757,  0.01493757,  0.01493757,
          0.01493757, -0.08962544,  0.01493757],
        [-0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ]],

       [[ 0.02857143,  0.02857143,  0.02857143,  0.02857143,
         -0.17142857,  0.02857143,  0.02857143],
        [ 0.02614816,  0.02614816,  0.02614816,  0.02614816,
          0.02614816, -0.15688897,  0.02614816],
        [-0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [-0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ]]])

## LM Head Forwording

LM head คือ ตัวแปลงผลลัพธ์ที่ได้จาก transformer block (ที่ให้ output ออกมาเป็น hidden states ของแต่ละตำแหน่งในประโยค) 

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

$h_{last}$ คือ hidden state ของ token ตัวสุดท้ายที่ได้จาก stack ของ transformer 

In [4]:
d_model = 6
vocab_size = 7
V = vocab_size
T = 4


In [5]:
W_vocab = -rng.random((d_model, V), np.float64)
b_vocab = -rng.random((vocab_size), np.float64)
h = rng.random((T, d_model), np.float64)

print(h)

[[0.90287328 0.51138753 0.78753013 0.64607151 0.07887539 0.63725101]
 [0.85884438 0.15227126 0.69963247 0.94677834 0.80096993 0.8069769 ]
 [0.10480225 0.37519001 0.97924236 0.23437907 0.21519851 0.919837  ]
 [0.26651731 0.92362002 0.09602509 0.94144425 0.27825632 0.32776049]]


In [6]:
logits = h @ W_vocab + b_vocab
print(logits)

[[-1.89948922 -1.55877692 -3.10879028 -1.78651971 -2.18645325 -1.77456551
  -2.09117315]
 [-2.41764435 -1.68826848 -3.58103725 -2.02831554 -2.08754221 -2.26318985
  -2.2950206 ]
 [-1.78808379 -0.8844733  -3.06757396 -1.03381838 -1.84196778 -1.85295169
  -2.23562877]
 [-1.03870891 -1.15464133 -3.04540924 -1.63096142 -1.21872928 -1.93752342
  -1.73831652]]


### In case of many batch

In [7]:
B = 2 # batch size

In [8]:
h = np.random.rand(B, T, d_model)
print (h.shape)

(2, 4, 6)


In [9]:
W_vocab.shape

(6, 7)

In [10]:
h[:, -1, :].shape

(2, 6)

In [11]:
logits = (h[:, -1, :] @ W_vocab) + b_vocab
print("logits => \n")
print(logits)

logits => 

[[-1.15995267 -1.03562607 -1.47927065 -1.347691   -1.29501931 -1.03374491
  -1.07223666]
 [-1.97582917 -1.31367731 -3.65398865 -1.67790846 -2.14326776 -2.16573326
  -2.63742219]]


## Back propagation of LM head

from the last chapter (loss And Softmax)

เราทราบว่า

$$\frac {\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} = \frac {\partial \mathcal {L}} {\partial Z} \in R^{B \times T \times V}$$

หรือมีมิติเท่ากับ logits

เมื่อ 
* $\mathcal {L}$ คือ loss ของ model
* $p$ คือ ความน่าจะเป็นของแต่ละคำที่จะออกเป็น token ถัดไป 
* $Z$ คือ Logits ที่ได้จากการคำนวณด้วย LM Head

In [12]:
# dZ = np.random.rand(B, T, V)
print("dZ =>", dZ.shape)

dZ => (2, 4, 7)


### Start from math
---

#### เมื่อต้องการปรับค่า $W_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 
$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial W_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial W_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial W_{vocab}}$$

จากฟังค์ชัน LM Head สำหรับ linear layer :

$$ Z = hW + b$$

gradient ของ $W$ ต้องเป็น "ผลรวมของ output product":
* ถ้ากรณีของ **last token only** :
$$dW = h_{last}^\intercal dZ$$

* ถ้ากรณีของทุก token :
$$ dW = \sum_{b, t} h_{b, t}^\intercal dZ_{b, t}$$


$$ \frac{\partial Z} {\partial W_{vocab}} = h_{last\ token}^\intercal$$

เมื่อ 
* $h \in \mathbb{R}^{B \times T \times d}$
* $dZ \in \mathbb{R}^{B \times T \times V}$
* $dW \in \mathbb{R}^{d \times V}$

#### เมื่อต้องการปรับค่า $b_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial b_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial b_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial b}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial b_{vocab}} = 1$$

ได้ว่า

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z}$$

แต่โดยปรกติแล้วเราจะใช้วิธีคำนวณแบบคิดทุก token เนื่องจาก 
* ถ้าคิดเฉพาะ last token &rarr; ได้สัญญาณสอน 1 จุดต่อ 1 sequence ทำให้เรียนช้ามาก ๆ
* คิดทุก token &rarr; ได้สัญญาณสอน ~T-1 จุด ต่อ sequence ซึ่งทำให้ใช้ข้อมูลได้คุ้มค่ากว่า


ตอน train เราทำ teacher forcing : ทุกตำแหน่งใน sequence มี target ของตัวเอง จาก (forward side) &rarr;
* $h \in \mathbb{R} ^ {B \times T \times d}$
* $logits = h\ @\ W_{vocab} + b_{vocab} \in \mathbb{R} ^ {B \times T \times V}$



Backward propagation ของ LM Head :
มีลำดับการไหลย้อนกลับดังนี้

**loss &rarr; logits (dZ) &rarr; W, b &rarr; hidden (dh)**

เมื่อ forward คือ : 

$$Z_{b, t, V} = h_{b, t, d} W + b$$

1. เราต้องมี gradient of logits ก่อน

ถ้า loss เป็น softmax + cross entropy แบบมาตรฐาน เราจะได้ว่า

$$\frac {\partial \mathcal{L}}{\partial Z} = dZ = \frac {P - Y} {N}$$

Where
* $P$ is softmax(logits) $P \in \mathbb {R} ^ {B, T, V}$
* $Y$ is one-hot of target $Y \in \mathbb {R} ^ {B, T, V}$
* $N$ is token count in loss after mask

2. **from $dZ$ to $dW, db$**

$$\frac {\partial \mathcal {L}} {\partial W} = dW = \sum_{b, t} h_{b, t}^\intercal dZ_{b, t}$$

$$\frac {\partial \mathcal {L}} {\partial b} = db = \sum_{b, t} dZ_{b, t}$$

3. **find $dh$**

from chain rule :
$$ \frac {\partial \mathcal {L}} {\partial h} = \frac {\partial \mathcal {L}} {\partial Z} \cdot \frac {\partial Z} {\partial h} = dh$$

$$dh_{b, t} = dZ_{b, t} W^\intercal$$

### Code demo
---

In [13]:
alpha = 0.1


เนื่องจากการดำเนินการ 3 มิติทำได้ยาก วิธีที่น่าจะดีกว่าคือการรวมมิติของ batch และ token เข้าด้วยกัน (กลายเป็น 2 มิติโดยการ)

In [14]:
print("shape of h before reshape: ", h.shape)
h = h.reshape(B * T, d_model)
print("shape of h after reshape: ", h.shape)

shape of h before reshape:  (2, 4, 6)
shape of h after reshape:  (8, 6)


In [15]:
print("shape of dZ before reshape: ", dZ.shape)
dZ = dZ.reshape(B * T, V)
print("shape of dZ after reshape : ", dZ.shape)

shape of dZ before reshape:  (2, 4, 7)
shape of dZ after reshape :  (8, 7)


In [16]:
dW = h.T @ dZ
print("now shape of dW will be (d, V) => ", dW.shape)
print("must be equal to shape of W_vocab =>", W_vocab.shape)

now shape of dW will be (d, V) =>  (6, 7)
must be equal to shape of W_vocab => (6, 7)


In [17]:
db = np.sum(dZ, axis=0)
print("now shape of db will be (V, ) => ", db.shape)
print("same as dW, db shape must be the same as b_vocab", b_vocab.shape)

now shape of db will be (V, ) =>  (7,)
same as dW, db shape must be the same as b_vocab (7,)


In [18]:
dh = dZ @ W_vocab.T
print("current shape of dh will be (B * T, d_model) => ", dh.shape)

dh = dh.reshape(B, T, d_model)
print("then reshape dh back to (B, T, d_model) => ", dh.shape)

current shape of dh will be (B * T, d_model) =>  (8, 6)
then reshape dh back to (B, T, d_model) =>  (2, 4, 6)


In [19]:
dh

array([[[ 0.0047309 , -0.02851535, -0.13561962,  0.07165096,
          0.01057472, -0.03511244],
        [ 0.04032968, -0.01704817,  0.07002992, -0.04101618,
         -0.05707668, -0.03169758],
        [-0.02885697,  0.04073668, -0.04297836, -0.02013535,
          0.04352404,  0.03589717],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]],

       [[ 0.05022935, -0.02123297,  0.08722008, -0.05108438,
         -0.07108724, -0.03947835],
        [-0.05051402,  0.07130938, -0.07523346, -0.03524686,
          0.07618864,  0.06283784],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]]])

In [20]:
h = h.reshape(B, T, d_model)
print(h.shape)

(2, 4, 6)


In [21]:
print(h)

[[[0.37386205 0.38874341 0.1103183  0.09438683 0.70366247 0.62612888]
  [0.9333875  0.79033129 0.6423229  0.23130628 0.24022694 0.37777587]
  [0.62945831 0.52483719 0.89377407 0.58787346 0.87098127 0.2364442 ]
  [0.90116807 0.00907852 0.15348103 0.03149705 0.16859034 0.32471581]]

 [[0.42861427 0.62378966 0.81627692 0.42015638 0.69268086 0.11851873]
  [0.45048331 0.27074882 0.04304258 0.47848674 0.99750007 0.9641103 ]
  [0.17413423 0.44526538 0.17293114 0.46778017 0.7577158  0.39567209]
  [0.38953835 0.74080766 0.98122062 0.78186992 0.49464442 0.42406262]]]
